<a href="https://colab.research.google.com/github/hungryphobic/NLP-projects/blob/main/P2_Twitter_Sentiment_Analysis_Malay_Language_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Import Module and package
---

In [ ]:
import pandas as pd
import re
import pickle
import numpy as np
from csv import reader
import nltk
from nltk.corpus import stopwords

# 2) Load Collected Raw Data
---

> The data topic and information details can be refered on Part 1 -3.1 of this project.

In [ ]:
dfKG = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/1_KerajaaanGagal_April29.csv')
dfMSSP = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/2_MakeSchoolASaferPlace_April29.csv')
dfKN = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/3_KRINanggala402_April29.csv')
dfSH = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/4_sinarharian_April29.csv')
dfCM = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/5_COVIDMalaysia_April29.csv')
dfKJK = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/6_KitaJagaKita_April29.csv')
dfJG = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/7_JusticeforGanapathy_April29.csv')
dfUPSR = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/8_UPSR_April29.csv')
dfARN = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tweets_data/9_AstroRadioNews_April29.csv')

### Test sentence string
> * This few sample test is use just to test our model before proceed with raw dataset, to detect failure in line of code and etc...
* The sample test text was selected from extracted raw dataset. the sample est should include all the main concern for our preprocessing purpose. It should contains noisy text (Uppercase, mispell word, @mention tag, #hashtag, emoji/emoticon, stopwords, Non-Malay word and etc...)

In [ ]:
TS1 = '@Marina_Ibrahims Kebabian PN makin terserlah dan muhyiddin kimak \n#KerajaanGagal' #contain ,#tag, @mention, Uppercase extra character
TS2 = '@kuasasiswa you made my day!👏😂\nso .lma dh X rasa @ujbu57ubj_78BJ gmbira mcm ni 👍 \n#Keraja...' #contain misspell word, emoticon/emoji, English word
TS3 = "@Izzud1n Omg 😭🤚🏼 1234 like, if we doesn't even get good results for upsr. not that we couldn't do well in sekolah menengah. aigoo https://t.co/K4GiA9l1Iw" #contain link, emoticon/emoji, English word, 

# 3) Data PreProcessing
---

## 3.1) Overall idea

> For this project purpose I will be covered 2 procesing tools which is using re- Regular Expression (standard python module for handling both pattern and string) *and combination of other techniques*.This module provides regular expression matching operations similar to those found in Perl. more info check - https://docs.python.org/3/library/re.html.

> Next is just PreProcesisng using **Malaya** nltk .which is a toolkit that known for processing Malay Language. **Malaya** is a Natural-Language-Toolkit library for bahasa Malaysia, powered by Deep Learning Tensorflow. More info check - https://malaya.readthedocs.io/en/latest/index.html#

## 3.2) PreProcessing 1- reMixin

###The Strategy/Methodology
---
> reMixin preprocesing is my propose preprocessing step which the overall step is combination of RE module and other other technique; as follows:
* Lowercasing the character --using python string manipulation
* Removing :
  * #Hashtag;i.e) #KerajaanGagal
  * @Mention or/@user; i.e) @mohd25
  * link; i.e) https://t.co/iCT2wJuiNLy6uiwENsx3
* 2 option for emoji and emoticon either:
  * 1) Remove 
  * 2) Translate them into text (Preferable to maintain the information)
* Identify English word and Translate them to Malay word -- using Malaya transformer translation
* Spelling Correction using Malaya probability spelling correction tools
* Removing stopwards and removing shortform word ( because most of them is stopwords and does not really affect the for clasification) i.e; yg == yang; utk == untuk; Except following shortform:
  * 'x', 'tk'  == 'tak' or tidak' whcih means 'no'
  * 'takda' or 'xda' == ' tak ada' or 'tidak ada' means ' do not have'.
* Removing the NonAlphanumeric and special character -re
  


### ***MusSpellingCorrection***
> user defined function. my own method of handling shortform word case in noisy malay social media text

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/')
from MusSpellingCorrection import pembetulEjaan

### ***Translate English to Malay(Bahasa Melayu)*
> I just using pretrained transformer model from Malaya nltk 

In [ ]:
%%time
!pip install malaya-gpu
import malaya

/usr/local/lib/python3.7/dist-packages/malaya/function/__init__.py:37: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  'Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0'


CPU times: user 3.56 s, sys: 388 ms, total: 3.95 s
Wall time: 1min 15s


___

In [ ]:
malaya.translation.en_ms.available_transformer()

INFO:root:tested on 77k EN-MS sentences.


,Size (MB),Quantized Size (MB),BLEU,Suggested length
small,42.7,13.4,0.512,256.0
base,234.0,82.7,0.696,256.0
large,817.0,244.0,0.699,256.0
bigbird,246.0,63.7,0.551,1024.0
small-bigbird,50.4,13.1,0.522,1024.0


#### *stripNonAlphaNumeric(string)*

In [ ]:
transformer = malaya.translation.en_ms.transformer()
transformer_small = malaya.translation.en_ms.transformer(model = 'small')
transformer_large = malaya.translation.en_ms.transformer(model = 'large')

downloading frozen model to /root/Malaya/translation-en-ms/base/model.pb


223MB [00:06, 32.6MB/s]                         


downloading frozen vocab to /root/Malaya/translation-en-ms/base/en-ms.subwords


1.00MB [00:00, 17.4MB/s]                   


downloading frozen model to /root/Malaya/translation-en-ms/small/model.pb


...

1.00MB [00:00, 13.2MB/s]                   


downloading frozen model to /root/Malaya/translation-en-ms/large/model.pb


100%|██████████| 778/778 [00:32<00:00, 23.8MB/s]
  0%|          | 0.00/0.22 [00:00<?, ?MB/s]

downloading frozen vocab to /root/Malaya/translation-en-ms/large/en-ms.subwords


1.00MB [00:00, 14.7MB/s]                   


In [ ]:
def stripNonAlphaNum(text):
  text = re.compile(r'\W+', re.UNICODE).split(text)   
  return ' '.join(text)

#### *StripHashTag(String)*

In [ ]:
def stripHashTag(text):
  text = re.compile(r'#\S+', re.UNICODE).split(text)   
  return ' '.join(text)

#### *StripMention(String)*

In [ ]:
def stripMention(text):
  text = re.compile(r'@\S+', re.UNICODE).split(text)   
  return ' '.join(text)

#### *StripHyperLink(String)*

In [ ]:
def stripHyperLink(text):
  text = re.compile(r'https?:\/\/\S+', re.UNICODE).split(text)   
  return ' '.join(text)

#### *stripUnderscore(String)*

In [ ]:
def stripUnderscore(text):
  text = re.sub(r'_', ' ', text)
  return text

#### *remove_emoji(string)*

In [ ]:
def removeEmoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" # emoticons
                           u"\U0001F300-\U0001F5FF" # symbols & pictographs
                           u"\U0001F680-\U0001F6FF" # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

#### *emojiToText(string)*

In [ ]:
#Convert/translate emoji to text
with open('/content/drive/MyDrive/Colab Notebooks/Emoji_Dict.p', 'rb') as fp:
    Emoji_Dict = pickle.load(fp)
    Emoji_Dict = {v: k for k, v in Emoji_Dict.items()}

def emojiToText(text):
    for emoj in Emoji_Dict:
      text = re.sub(r'('+emoj+')',Emoji_Dict[emoj], text)          
    return text

In [ ]:
Emoji_Dict

#### *removeStopWord(string)*

In [ ]:
stopWord = open('/content/drive/MyDrive/Colab Notebooks/MalayStopWord.csv', 'r').read().split()
def removeStopWord(text):
  text = text.split()
  text = [word for word in text if word not in stopWord]
  return ' '.join(text)


#### *pembetulEjaan(string)*
refer MusSpellingCorrection.py file

#### ***reMixin(string)***

In [ ]:
def reMixin(text):
  text = text.lower()
  text = emojiToText(text) 
  text = stripHashTag(text)
  text = stripMention(text)
  text = stripHyperLink(text)
  text = stripNonAlphaNum(str(text))
  text = stripUnderscore(text)
  text = transformer.beam_decoder([text])
  text = stripNonAlphaNum(str(text))
  text = pembetulEjaan(text)  
  text = removeStopWord(text)
  return text

In [ ]:
%%time
print(TS2)
print(reMixin(TS2))

@kuasasiswa you made my day!👏😂
so .lma dh X rasa @ujbu57ubj_78BJ gmbira mcm ni 👍 
#Keraja...
membuat hari bertepuk tangan air mata kegembiraan jadi tak rasa gembira thumbs up
CPU times: user 21.1 s, sys: 9.1 s, total: 30.2 s
Wall time: 23.3 s


### **PreProcesing using reMixin and export the clan data**

In [ ]:
%%time
dfKG['CD-RE'] = dfKG['Raw'].astype(str).apply(reMixin)

CPU times: user 13min 10s, sys: 2min 26s, total: 15min 36s
Wall time: 9min 33s


In [ ]:
%%time
dfKG.to_csv ('/content/drive/MyDrive/Colab Notebooks/tweets_data/clean dataset/1.csv', index = False, header=True)

CPU times: user 3.02 ms, sys: 811 µs, total: 3.83 ms
Wall time: 10.1 ms


In [ ]:
%%time
dfMSSP['CD-RE'] = dfMSSP['Raw'].astype(str).apply(reMixin)

CPU times: user 16min 41s, sys: 3min 1s, total: 19min 43s
Wall time: 12min 1s


In [ ]:
%%time
dfMSSP.to_csv ('/content/drive/MyDrive/Colab Notebooks/tweets_data/clean dataset/2.csv', index = False, header=True)

CPU times: user 2.52 ms, sys: 1.1 ms, total: 3.62 ms
Wall time: 7.87 ms


In [ ]:
%%time
dfKN['CD-RE'] = dfKN['Raw'].astype(str).apply(reMixin)

CPU times: user 17min 45s, sys: 3min 22s, total: 21min 7s
Wall time: 12min 57s


In [ ]:
%%time
dfKN.to_csv ('/content/drive/MyDrive/Colab Notebooks/tweets_data/clean dataset/3.csv', index = False, header=True)

CPU times: user 1.55 ms, sys: 1.92 ms, total: 3.47 ms
Wall time: 12.3 ms


In [ ]:
%%time
dfSH['CD-RE'] = dfSH['Raw'].astype(str).apply(reMixin)

CPU times: user 18min 17s, sys: 3min 45s, total: 22min 2s
Wall time: 13min 35s


In [ ]:
%%time
dfSH.to_csv ('/content/drive/MyDrive/Colab Notebooks/tweets_data/clean dataset/4.csv', index = False, header=True)

CPU times: user 3.13 ms, sys: 0 ns, total: 3.13 ms
Wall time: 8.49 ms


In [ ]:
%%time
dfCM['CD-RE'] = dfCM['Raw'].astype(str).apply(reMixin)

CPU times: user 2min 1s, sys: 28.8 s, total: 2min 30s
Wall time: 1min 34s


In [ ]:
%%time
dfCM.to_csv ('/content/drive/MyDrive/Colab Notebooks/tweets_data/clean dataset/5.csv', index = False, header=True)

CPU times: user 2.63 ms, sys: 98 µs, total: 2.73 ms
Wall time: 10.1 ms


In [ ]:
%%time
dfKJK['CD-RE'] = dfKJK['Raw'].astype(str).apply(reMixin)

CPU times: user 15min 6s, sys: 2min 53s, total: 17min 59s
Wall time: 11min 2s


In [ ]:
%%time
dfKJK.to_csv ('/content/drive/MyDrive/Colab Notebooks/tweets_data/clean dataset/6.csv', index = False, header=True)

CPU times: user 3.5 ms, sys: 858 µs, total: 4.36 ms
Wall time: 7.49 ms


In [ ]:
%%time
dfJG['CD-RE'] = dfJG['Raw'].astype(str).apply(reMixin)

CPU times: user 11min 6s, sys: 2min 2s, total: 13min 9s
Wall time: 8min 3s


In [ ]:
%%time
dfJG.to_csv ('/content/drive/MyDrive/Colab Notebooks/tweets_data/clean dataset/7.csv', index = False, header=True)

CPU times: user 3.04 ms, sys: 668 µs, total: 3.71 ms
Wall time: 9.13 ms


In [ ]:
%%time
dfUPSR['CD-RE'] = dfUPSR['Raw'].astype(str).apply(reMixin)

CPU times: user 21min 3s, sys: 4min 6s, total: 25min 10s
Wall time: 15min 25s


In [ ]:
%%time
dfUPSR.to_csv ('/content/drive/MyDrive/Colab Notebooks/tweets_data/clean dataset/8.csv', index = False, header=True)

CPU times: user 2.55 ms, sys: 880 µs, total: 3.43 ms
Wall time: 10.2 ms


In [ ]:
%%time
dfARN['CD-RE'] = dfARN['Raw'].astype(str).apply(reMixin)

CPU times: user 17min 16s, sys: 3min 22s, total: 20min 38s
Wall time: 12min 41s


In [ ]:
%%time
dfARN.to_csv ('/content/drive/MyDrive/Colab Notebooks/tweets_data/clean dataset/9.csv', index = False, header=True)

CPU times: user 3.16 ms, sys: 610 µs, total: 3.77 ms
Wall time: 8.66 ms
